In [1]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [2]:
import os
import random
import PIL
import numpy as np # linear algebra
import matplotlib.pyplot as plt # Ploting the diagrams

from sklearn.utils import shuffle

import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D

import os
from os import listdir
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt # Ploting the diagrams
import matplotlib.image as mpimg


from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import load_img
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from tensorflow.keras import models
import random
from keras.models import model_from_json
import pandas as pd
tf.random.set_seed(333)

## 1. Initialize

In [3]:
clients = 5
rounds = 12
splits = (clients+1)*rounds+1

## 2. Loading dataset in memory

In [4]:
def genImageList(l):
    return list(filter(lambda i: i.split(".")[-1] in ["png","jpg","jpeg"], l))
def genDf(images,labels):
    df= pd.DataFrame(images,columns=["Image"])
    df["Class"] = labels
    return df

## 2.1 Generating Global Dataset

In [5]:
#Getting all FileNames
gMaskPath = r'Test/Mask/'
gNoMaskPath = r'Test/NoMask/'
gMaskImages = genImageList([gMaskPath+i for i in os.listdir(gMaskPath)])
gMaskLabels = np.zeros(len(gMaskImages))
gNoMaskImages = genImageList([gNoMaskPath+i for i in os.listdir(gNoMaskPath)])
gNoMaskLabels = np.ones(len(gNoMaskImages))

gMaskDf = genDf(gMaskImages,gMaskLabels)
gNoMaskDf = genDf(gNoMaskImages,gNoMaskLabels)

gDf = pd.DataFrame(shuffle(pd.concat([gMaskDf,gNoMaskDf])),columns=gMaskDf.columns)

## 3. Stratified Kfolds Custom Code

In [6]:
def genKSplitDf(df,splits):
    index = np.array_split(shuffle(df.index,random_state=3333),splits)    
    folds = []
    for i in range(splits):
        folds.append(df.filter(items = index[i], axis=0))
    return folds
def mergeDf(df1,df2,splits):
    columns = df1[0].columns
    folds = []
    for i in range(splits):
        folds.append(pd.DataFrame(shuffle(pd.concat([df1[i],df2[i]]),random_state=3333),columns=columns)) 
    return folds

In [7]:
maskFolds = genKSplitDf(gMaskDf,splits)
noMaskFolds = genKSplitDf(gNoMaskDf,splits)
folds = mergeDf(maskFolds,noMaskFolds,splits)

In [8]:
len(folds)

73

## 4. Simulation Setup for Federated Learning

In [8]:
def plot_history(history):
    
    fig, ax = plt.subplots(1, 1,figsize=(5,5))
    ax.plot(history.history['accuracy'])
    ax.plot(history.history['val_accuracy'])
    ax.set_title('Model Accuracy')
    ax.set(xlabel='Epoch',ylabel='Accuracy')
    ax.legend(['train', 'test'], loc='upper left')

def createModel(shape=(100,100,3)):
    tf.random.set_seed(3333)
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  # loss='kl_divergence',
                  loss = 'binary_crossentropy',
                  metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(),tf.keras.metrics.AUC(
        num_thresholds=200, curve='ROC',
        summation_method='interpolation', name=None, dtype=None,
        thresholds=None, multi_label=False, label_weights=None
    ),'accuracy'])
    
    return model


def genModel(df,model,name):
    
    x,y = df["Image"],df["Class"]
    data = []
    size = 100
    
    for img_path in x:
        image = load_img(img_path, target_size=(size, size))
        image = img_to_array(image)
        image = image/255
        data.append(image)

    data = np.array(data, dtype="float32")
    labels = np.array(y).reshape([-1,1])
    
    # trainx,testx,trainy,testy = train_test_split(data,labels,random_state=333,test_size=0.3)
    
    history = model.fit(data, labels, epochs=10,validation_split=0.2,verbose=0)
    acc = []
    loss = []
    
    for i,j in zip(history.history["loss"],history.history["accuracy"]):
        acc.append(j)
        loss.append(i)
    
    # plot_history(history,name)
    return model,acc,loss

## 4.1 Initializing Global Model

In [10]:
globalModel,initAcc,initLoss = genModel(folds.pop(),createModel(),"Init")

## Initializing Cleint Models

In [9]:
def genClientModel(globalModel,name):
    model = createModel()
    model.set_weights(globalModel.get_weights())
    model._name = name
    return model

def preprocessWeights(weights,acc):
    newWeights = []
    for i,j in zip(weights,acc):
        temp = []
        for w in i:
            temp.append(w*j)
        newWeights.append(temp)
    return newWeights

def avergeWeights(w):
    acc = np.array(w[0],dtype=object)
    for i in range(1,len(w)):
        acc = acc + np.array(w[i],dtype=object)
    acc = acc/len(w)
    return [np.array(i) for i in acc.tolist()]
    
def updateWeights(newWeights,model,layers):
    upModels = []
    
    for m in model:
        updated = []
        
        w = m.get_weights()
        
        for l in range(len(w)):
            if l in layers:
                updated.append(newWeights[l])
            else:
                updated.append(w[l])
        
        m.set_weights(updated)
        upModels.append(m)
        
    return upModels

In [12]:
#Based on our model
shallow = [0,1,2,3,4,5]
deep = [6,7,8,9]
allLayers =  shallow + deep

In [13]:
allLayers

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [14]:
clientModels = [genClientModel(globalModel,"Client_"+str(i)) for i in range(clients)]
modelRes = np.zeros(clients).tolist()

clientRes = {}
clientLossHist = {}
clientAccHist = {}
for c in range(clients):
    clientRes[c] = []
    clientLossHist[c] = initLoss[:]
    clientAccHist[c] =  initAcc[:]

In [15]:
for i in range(rounds):
    print("Round - ",i+1)
    
    for c in range(clients):
        clientModels[c],acc,loss = genModel(folds.pop(),clientModels[c],"C"+str(c)+"_"+str(i)) 
        clientLossHist[c].extend(loss[:])
        clientAccHist[c].extend(acc[:])  
            
    w = preprocessWeights([model.get_weights() for model in clientModels],[1 for res in modelRes])
    avgW = avergeWeights(w)

    clientModels = updateWeights(avgW,clientModels,allLayers)
    

Round -  1


C:\Users\yashb\anaconda3\envs\goku\lib\site-packages\PIL\Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Round -  2
Round -  3
Round -  4
Round -  5
Round -  6
Round -  7
Round -  8
Round -  9
Round -  10
Round -  11
Round -  12


In [11]:
central,acc,loss = genModel(gDf,createModel(),"")
central.save("model.h5")

In [17]:
def genXY(df):
    x,y = df["Image"],df["Class"]
    data = []
    size = 100
    
    for img_path in x:
        image = load_img(img_path, target_size=(size, size))
        image = img_to_array(image)
        image = image/255
        data.append(image)
    data = np.array(data, dtype="float32")
    labels = np.array(y).reshape([-1,1])
    return data,labels
gMaskPath = r'Global/Mask/'
gNoMaskPath = r'Global/NoMask/'
gMaskImages = genImageList([gMaskPath+i for i in os.listdir(gMaskPath)])
gMaskLabels = np.zeros(len(gMaskImages))
gNoMaskImages = genImageList([gNoMaskPath+i for i in os.listdir(gNoMaskPath)])
gNoMaskLabels = np.ones(len(gNoMaskImages))

gMaskDf = genDf(gMaskImages,gMaskLabels)
gNoMaskDf = genDf(gNoMaskImages,gNoMaskLabels)

tDf = pd.DataFrame(shuffle(pd.concat([gMaskDf,gNoMaskDf])),columns=gMaskDf.columns)

tx,ty = genXY(tDf)

In [18]:
res = []
for c in range(clients):
    res.append(clientModels[c].evaluate(tx,ty)[-1])

188/188 [==============================] - 1s 6ms/step - loss: 0.3978 - recall_5: 0.9850 - precision_5: 0.8819 - auc_5: 0.9598 - accuracy: 0.9265


In [19]:
df = pd.DataFrame()
df["Testing Results"] = res
df.to_csv("Results/v.csv")

In [20]:
dfL = pd.DataFrame()
for k in clientLossHist.keys():
    dfL["Client "+ str(k)] = clientLossHist[k]
dfL.to_csv("vLoss.csv",index=False)

In [21]:
df = pd.DataFrame()
for k in clientLossHist.keys():
    df["Client "+ str(k)] = clientAccHist[k]
df.to_csv("Results/vAcc.csv",index=False)

In [22]:
central.evaluate(tx,ty)

188/188 [==============================] - 1s 6ms/step - loss: 0.2186 - recall_6: 0.9803 - precision_6: 0.9206 - auc_6: 0.9801 - accuracy: 0.9479


[0.21864436566829681,
 0.9802939295768738,
 0.9206398725509644,
 0.9800945520401001,
 0.9478957653045654]